In [1]:
from qaoa import *

In [2]:
# Example 1: Creating an Ising Hamiltonian and Finding Fisher Information Matrix

# Define the number of qubits and connectivity type ("open" or "closed")
n_qubits = 4
connectivity = "closed"

# Create an Ising Hamiltonian
ising_hamiltonian = H_zz_Ising(n_qubits, connectivity)


# Initialize QAOA with a specific depth and the Ising Hamiltonian
qaoa = QAOA(depth=1, H=ising_hamiltonian)

# Define a set of parameters
angles = [0.5, 0.2]  # Adjust these parameters as needed

# Calculate the Fisher Information Matrix
qfi_matrix = qaoa.qaoa_qfi_matrix(angles, state_ini=plus_state(n_qubits)) #|+>

print("Fisher Information Matrix:")
with np.printoptions(precision=3, suppress=True):
    print(qfi_matrix)


Fisher Information Matrix:
[[16.     0.   ]
 [ 0.    29.273]]


In [5]:
# Example 2: Creating a k-SAT Hamiltonian and Optimizing it with QAOA

# Define the number of variables (n), clauses (k), and alpha for k-SAT Hamiltonian
n = 5
k = 3
alpha = 2.0

# Create a k-SAT Hamiltonian
k_sat_hamiltonian = H_sat(n, k, alpha)

# Initialize QAOA with a specific depth and the k-SAT Hamiltonian
qaoa = QAOA(depth=2, H=k_sat_hamiltonian)


# Run the QAOA optimization using the L-BFGS-B method
qaoa.run_RI()

# Print the optimization results
with np.printoptions(precision=3, suppress=True):
    print("QAOA Parameters:", qaoa.opt_angles)
    print("Optimized QAOA Energy:", qaoa.q_energy)


QAOA Parameters: [5.367 3.89  1.944 1.518]
Optimized QAOA Energy: 0.33604863719537703


In [11]:
# Example 3: Solving the Max Cut Problem

# Define the graph adjacency matrix (replace this with your specific graph)
graph = [
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 1, 0]
]

# Number of qubits, which is the same as the number of nodes in the graph
num_qubits = len(graph)

# Convert the graph into the corresponding Ising Hamiltonian
max_cut_hamiltonian = Graph_to_Hamiltonian(graph, num_qubits)

# Now you can use your QAOA solver to find the maximum cut
qaoa = QAOA(depth=4, H=max_cut_hamiltonian)
qaoa.run_RI()

# Extract the results
max_cut_solution = qaoa.f_state  # The quantum state
max_cut_value = qaoa.q_energy  # The value of the maximum cut

# Output the results
with np.printoptions(precision=3, suppress=True):
    print("Max Cut Solution (quantum state):\n", max_cut_solution)
    print("Max Cut Value:", max_cut_value)

Max Cut Solution (quantum state):
 [ 0.  +0.j     0.  +0.j     0.  +0.j    -0.  -0.j     0.  +0.j
 -0.  -0.j    -0.42+0.569j  0.  +0.j     0.  +0.j    -0.42+0.569j
 -0.  -0.j     0.  +0.j    -0.  -0.j     0.  +0.j     0.  +0.j
  0.  +0.j   ]
Max Cut Value: -3.9999999999286837
